In [ ]:
# Importação dos módulos necessários

import mappings
import models
import queries
import utils

import geopandas as gpd
import h3
import logging
import networkx as nx
import osmnx as ox
import pandas as pd
import shapely
import sqlalchemy
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
# Conexão com o banco de dados

EXEC_MODO_TESTES = True
dados_conexao = models.ConexaoBD(EXEC_MODO_TESTES)

engine = sqlalchemy.create_engine(f'postgresql://{dados_conexao.user}:{dados_conexao.password}@{dados_conexao.server}:{dados_conexao.port}/{dados_conexao.database}')
conexao_bd = engine.connect()

In [ ]:
# Criação das tabelas no banco de dados

try :
    conexao_bd.execute(queries.CREATE_TABELA_UF)
    conexao_bd.execute(queries.CREATE_TABELA_MUNICIPIO)
    conexao_bd.execute(queries.CREATE_TABELA_MODALIDADE_TRANSPORTE)
    conexao_bd.execute(queries.CREATE_TABELA_GEOMETRIA_MUNICIPIO)
    conexao_bd.execute(queries.CREATE_TABELA_PONTO_INTERESSE)
    conexao_bd.execute(queries.CREATE_TABELA_MALHA_HEXAGONAL)
    conexao_bd.execute(queries.CREATE_TABELA_AREA_ANALISE)
    conexao_bd.execute(queries.CREATE_TABELA_GRAFO_NOS)
    conexao_bd.execute(queries.CREATE_TABELA_GRAFO_ARESTAS)
except Exception as e:
    logging.exception(msg='Erro inesperado ao criar tabelas no banco de dados')

In [ ]:
# Carga inicial dos dados

def realizar_carga_inicial():
    df_tabela_dtb = pd.read_csv('../data/tabela_dtb.csv')

    df_unidades_federativas = df_tabela_dtb.loc[:, ['UF', 'Nome_UF']].drop_duplicates()
    df_unidades_federativas.rename(columns={'UF':'codigo', 'Nome_UF':'nome'}, inplace=True)

    df_municipios = df_tabela_dtb.loc[:, ['Código Município Completo', 'Nome_Município', 'UF']]
    df_municipios.rename(columns={'Código Município Completo':'codigo', 'Nome_Município':'nome', 'UF':'codigo_uf'}, inplace=True)

    df_modalidades_transporte = pd.DataFrame(data=[['walk', 3.6], ['bike', 13]], columns=['nome', 'velocidade_media_kph'])

    # Persistência no banco de dados

    df_unidades_federativas.to_sql(name='t_unidade_federativa', con=conexao_bd, schema='public', if_exists='append', index=False)
    df_municipios.to_sql(name='t_municipio', con=conexao_bd, schema='public', if_exists='append', index=False)
    df_modalidades_transporte.to_sql(name='t_modalidade_transporte', con=conexao_bd, schema='public', if_exists='append', index=False)

In [ ]:
# Extração dos polígonos dos municípios e seus pontos de interesse

def extrair_dados_base_municipios():
    df_municipios = pd.read_sql(sql=queries.SELECT_MUNICIPIOS, con=conexao_bd)

    if EXEC_MODO_TESTES:
        df_municipios = df_municipios(df_municipios['codigo'].isin([1100015, 3104007, 3106200, 3304557, 3550308]))

    for idx in tqdm(iterable=df_municipios.index, desc='Buscando dados básicos dos municípios'):
        df_geometria_municipio = pd.DataFrame(data=[], columns=['geometria', 'codigo_municipio'])
        df_pontos_interesse_municipio = pd.DataFrame(data=[], columns=['tipo', 'geometria', 'codigo_municipio'])

        municipio = models.Municipio(df_municipios.loc[idx, :])
        query_osmnx = utils.montar_query_osmnx(municipio)

        try:
            gdf_geometria = ox.geocode_to_gdf(query=query_osmnx)
            gdf_pontos_interesse = ox.features_from_place(query=query_osmnx, tags={'amenity':True})

            # TODO Aplicar filtragem dos pontos de interesse relevantes por tipo
        except Exception as e:
            logging.exception(msg=f'Erro ao consultar os dados básicos do município {municipio.codigo} - {municipio.nome}')
            continue
        
        df_geometria_municipio.loc[len(df_geometria_municipio)] = [
            gdf_geometria['geometry'][0],
            municipio.codigo
        ]

        for idx_p in gdf_pontos_interesse.index:
            df_pontos_interesse_municipio.loc[len(df_pontos_interesse_municipio)] = [
                gdf_pontos_interesse['amenity'][idx_p],
                gdf_pontos_interesse['geometry'][idx_p].centroid,
                municipio.codigo
            ]
        
        gdf_geometria_municipio = gpd.GeoDataFrame(data=df_geometria_municipio, geometry='geometria', crs='EPSG:4326')
        gdf_pontos_interesse_municipio = gpd.GeoDataFrame(data=df_pontos_interesse_municipio, geometry='geometria', crs='EPSG:4326')

        gdf_geometria_municipio.to_postgis(name='t_geometria_municipio', con=conexao_bd, schema='public', if_exists='append', index=False)
        gdf_pontos_interesse_municipio.to_postgis(name='t_ponto_interesse', con=conexao_bd, schema='public', if_exists='append', index=False)

In [ ]:
# Geração da malha hexagonal dos municípios

def gerar_malhas_hexagonais():
    gdf_municipios = gpd.read_postgis(sql=queries.SELECT_GEOMETRIA_MUNICIPIOS, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

    for idx in tqdm(iterable=gdf_municipios.index, desc='Gerando malhas hexagonais dos municípios'):
        codigo_municipio = gdf_municipios['codigo_municipio'][idx]
        df_malha_hexagonal = pd.DataFrame(data=[], columns=['codigo', 'geometria', 'codigo_municipio'])

        for poligono in list(gdf_municipios['geometria'][idx].geoms):
            coords_municipio = poligono.exterior.coords
            try:
                hexagonos = h3.polyfill_polygon(outer=coords_municipio, res=8)

                for hex in hexagonos:
                    geometria_hex = shapely.Polygon(h3.h3_to_geo_boundary(hex))
                    
                    df_malha_hexagonal.loc[len(df_malha_hexagonal)] = [
                        hex,
                        geometria_hex,
                        gdf_municipios['codigo_municipio'][idx]
                    ]
            except Exception as e:
                logging.error(msg=f'Erro ao gerar a malha hexagonal do município {codigo_municipio}')
                continue

        gdf_malha_hexagonal = gpd.GeoDataFrame(df_malha_hexagonal, geometry='geometria', crs='EPSG:4326')
        gdf_malha_hexagonal.to_postgis(name='t_malha_hexagonal', con=conexao_bd, schema='public', if_exists='append', index=False)

In [ ]:
# Geração das áreas de análise

def gerar_areas_analise():
    df_municipios = pd.read_sql(sql=queries.SELECT_MUNICIPIOS_COM_MALHA, con=conexao_bd)
    df_modalidades_transporte = pd.read_sql(sql=queries.SELECT_MODALIDADES_TRANSPORTE, con=conexao_bd)

    for idx in tqdm(iterable=df_municipios.index, desc='Gerando as áreas de análise dos municípios'):
        codigo_municipio = df_municipios['codigo'][idx]
        df_area_analise = pd.DataFrame(data=[], columns=['geometria', 'codigo_hexagono', 'codigo_municipio', 'codigo_modalidade_transporte'])

        query = utils.montar_query_parametrizada(sql=queries.SELECT_MALHA_MUNICIPIO, params={'codigo_municipio':codigo_municipio})
        gdf_malha_hexagonal = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

        query = utils.montar_query_parametrizada(sql=queries.SELECT_PONTOS_INTERESSE_MUNICIPIO, params={'codigo_municipio':codigo_municipio})
        gdf_pontos_interesse = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

        gdf_malha_hexagonal.to_crs('EPSG:3857', inplace=True)
        gdf_pontos_interesse.to_crs('EPSG:3857', inplace=True)

        for idx_m in df_modalidades_transporte.index:
            raio_analise = df_modalidades_transporte['velocidade_media_kph'][idx_m] * 15/60 * 2 * 1000

            for idx_h in gdf_malha_hexagonal.index:
                geo_area_analise = gdf_malha_hexagonal.loc[idx_h, 'geometria'].centroid.buffer(raio_analise)

                if gdf_pontos_interesse.centroid.within(geo_area_analise).any():
                    df_area_analise.loc[len(df_area_analise)] = [
                        geo_area_analise,
                        gdf_malha_hexagonal['codigo'][idx_h],
                        codigo_municipio,
                        df_modalidades_transporte['codigo'][idx_m]
                    ]

        gdf_area_analise = gpd.GeoDataFrame(data=df_area_analise, geometry='geometria', crs='EPSG:3857').to_crs('EPSG:4326')
        gdf_area_analise.to_postgis(name='t_area_analise', con=conexao_bd, schema='public', if_exists='append', index=False)

In [ ]:
# Geração dos grafos da rede de transporte dos municípios

def buscar_grafos_municipios():
    df_municipios = pd.read_sql(sql=queries.SELECT_MUNICIPIOS_COM_MALHA, con=conexao_bd)
    df_modalidades_transporte = pd.read_sql(sql=queries.SELECT_MODALIDADES_TRANSPORTE, con=conexao_bd)

    for idx in tqdm(iterable=df_municipios.index, desc='Gerando os grafos da rede de transporte dos municípios'):
        municipio = models.Municipio(df_municipios.loc[idx, :])

        for idx_m in df_modalidades_transporte.index:
            modalidade_transporte = models.ModalidadeTransporte(df_modalidades_transporte.loc[idx_m, :])

            try:
                gph_rede_transporte = ox.graph_from_place(utils.montar_query_osmnx(municipio), network_type=modalidade_transporte.nome, simplify=True)
                gph_rede_transporte = ox.utils_graph.remove_isolated_nodes(gph_rede_transporte)
                nx.set_edge_attributes(gph_rede_transporte, values=modalidade_transporte.velocidade_media_kph, name='speed_kph')
                gph_rede_transporte = ox.add_edge_travel_times(gph_rede_transporte)
            except Exception as e:
                logging.exception(f'Erro ao buscar o grafo da rede de transporte do município {municipio.codigo} - {municipio.nome}')
                continue

            gdf_nodes, gdf_edges = ox.graph_to_gdfs(gph_rede_transporte)

            if gdf_nodes.empty or gdf_edges.empty:
                continue
            
            colunas_drop = gdf_nodes.columns.difference(other=mappings.GDF_PARA_TABELA_GRAFO_NO.keys())
            gdf_nodes.drop(columns=colunas_drop, inplace=True)
            gdf_nodes.index.rename(mappings.GDF_PARA_TABELA_GRAFO_NO.get('index'), inplace=True)
            gdf_nodes.rename(columns=mappings.GDF_PARA_TABELA_GRAFO_NO, inplace=True)
            gdf_nodes['codigo_municipio'] = municipio.codigo
            gdf_nodes['codigo_modalidade_transporte'] = modalidade_transporte.codigo
            gdf_nodes.set_geometry('geometria', inplace=True)

            colunas_drop = gdf_edges.columns.difference(other=mappings.GDF_PARA_TABELA_GRAFO_ARESTA.keys())
            gdf_edges.drop(columns=colunas_drop, inplace=True)
            gdf_edges.index.rename(mappings.GDF_PARA_TABELA_GRAFO_ARESTA.get('index'), inplace=True)
            gdf_edges.rename(columns=mappings.GDF_PARA_TABELA_GRAFO_ARESTA, inplace=True)
            gdf_edges['codigo_municipio'] = municipio.codigo
            gdf_edges['codigo_modalidade_transporte'] = modalidade_transporte.codigo
            gdf_edges.set_geometry('geometria', inplace=True)

            gdf_nodes.to_postgis(name='t_grafo_no', con=conexao_bd, schema='public', if_exists='append', index=True)
            gdf_edges.to_postgis(name='t_grafo_aresta', con=conexao_bd, schema='public', if_exists='append', index=True)

In [ ]:
# Execução das etapas de extração de dados

realizar_carga_inicial()
extrair_dados_base_municipios()
gerar_malhas_hexagonais()
gerar_areas_analise()
buscar_grafos_municipios()